In [2]:
#メモリのリセット
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [43]:
#データ準備
#元データの作り方
#所在管理MBからデータ作成
#納入日と回収日をフィルだリングする
pre_zaikodata = pd.read_csv('20231107使用_所在管理LT_納入日0828_0929_回収日8月9月.csv',encoding='shift_jis')
pre_zaikodata.head(5)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/29,...,2023/9/30 7:13,< NULL >,0.14,2.31,2.59,2.75,< NULL >,2.86,2023/9/30 7:13,< NULL >
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.79,< NULL >,3.86,2023/9/30 7:13,< NULL >
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.08,2.02,2.15,3.86,< NULL >,4.02,2023/9/30 7:13,< NULL >
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.76,< NULL >,3.86,2023/9/30 7:13,< NULL >
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023/9/27,...,2023/9/30 7:13,< NULL >,2.07,3.99,4.28,6.86,< NULL >,7.00,2023/9/30 7:13,< NULL >


In [44]:
#各LTの計算

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
        LT2 = float(pre_zaikodata.loc[i, '発注〜検収LT']) - float(pre_zaikodata.loc[i, '発注〜印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        pre_zaikodata.loc[i, '印刷検収LT'] = LT2
        pre_zaikodata.loc[i, '検収入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] = LT6
        count = count + 1
print(count)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
51857


In [17]:
pre_zaikodata['納入日2'].head()

0    9
1    9
2    9
3    9
4    9
Name: 納入日2, dtype: int64

In [47]:
###基準在庫枚数加味する
#文字型を整数型にする
#当月必要数？

pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['納入日（月）'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')

count = 0
tehaidata = pd.read_csv('20231107使用__202309_手配数_1Y_12次.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample3.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    #print(pre_zaikodata.loc[i, '回収日時'],pre_zaikodata.loc[i,'品番'] )
    if pre_zaikodata.loc[i, '納入日（月）'].month != 8:
        #print(pre_zaikodata.loc[i, '納入日（月）'].month)
        if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
            hinban = pre_zaikodata.loc[i,'品番'] 
            hinban2 = hinban.replace("-", "") 
            hinban3 = hinban2.replace(" ", "") 
            #print(i)
            if i % 5000==0:
                print(i)
            for j in range(len(long2)):
                #print(j)
                tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
                tehaihinban2 = tehaihinban.replace(' ', '')
                if hinban3 == tehaihinban2:
                    day = pre_zaikodata.loc[i,'回収日時'].day
                    index = tehaidata.columns.get_loc("翌々々月稼働日数") + day
                    #count = pre_zaikodata.groupby('品番')['かんばんシリアル'].count()
                    #pre_zaikodata.loc[i,'実かんばん数']　= 
                    #print(day,index,str(tehaidata.iloc[j,index]),str(tehaidata.iloc[j,index]).replace(',', ''))
                    pre_zaikodata.loc[i,'日量数'] = int(str(tehaidata.iloc[j,index]).replace(',', ''))
                    pre_zaikodata.loc[i,'日量数（箱数）'] = pre_zaikodata.loc[i,'日量数'] /pre_zaikodata.loc[i, '収容数']
                    if int(str(tehaidata.loc[j,'当月必要数']).replace(',', '')) != 0:
                        avenichiryo = int(str(tehaidata.loc[j,'当月必要数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                        pre_zaikodata.loc[i,'平均日量数'] = avenichiryo
                        pre_zaikodata.loc[i,'平均日量数（箱数）'] = avenichiryo//pre_zaikodata.loc[i, '収容数']
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                    else:
                        pre_zaikodata.loc[i,'平均日量数'] = 0
                        pre_zaikodata.loc[i,'平均日量数（箱数）'] = 0
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
    else:
        pre_zaikodata.loc[i,'平均日量数'] = 0
        pre_zaikodata.loc[i,'平均日量数（箱数）'] = 0
        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
print(count)

0
5000
15000
20000
25000
45000
0


In [48]:
count = 0
tehaidata = pd.read_csv('20231107使用_XR10_手配運用情報2300911_1Y（回収引当適用）.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample2.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注〜印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '便Ave'] = pre_zaikodata.loc[i,'日量数（箱数）']/pre_zaikodata.loc[i, 'B']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種類'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '設計かんばん数'] = pre_zaikodata.loc[i,'日量数（箱数）'] + pre_zaikodata.loc[i, '基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数（日数）'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT（検収〜回収LT）'] 
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0〜0.25倍'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0.25〜0.5倍'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常？：設計値の0.5〜0.75倍'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の0.75〜1倍'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の1〜1.25倍'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常：設計値の1.25〜1.5倍'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の1.5〜2倍'
                elif (zisseki >= MAX*2):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の2倍以上'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)

0 G1144ECE010    
5000 9036340A010    
10000 G9210ECE020    
15000 G1144ECE010    
20000 G1144ECB010    
25000 41310ECE010    
30000 35847ECE010    
35000 G9210ECE020    
40000 39171ECB010    
45000 35771ECE010    
50000 35771ECB010    
55000 41310ECE010    
58113


In [87]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_在庫推移9月.csv'

# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
new_kanban_data = pd.read_csv(new_file_path, encoding='cp932')

# '計測日時'を日時型に変換し、週番号を含む新しい列を追加します。
new_kanban_data['計測日時'] = pd.to_datetime(new_kanban_data['計測日時'], errors='coerce')
new_kanban_data.dropna(subset=['計測日時'], inplace=True)
new_kanban_data['Year'] = new_kanban_data['計測日時'].dt.year
new_kanban_data['Week_Number'] = new_kanban_data['計測日時'].dt.week

# '品番'と'Week_Number'でグループ化し、'在庫数（箱）', '入庫数（箱）', '出庫数（箱）'を合計します。
weekly_inventory_aggregated = new_kanban_data.groupby(['品番', 'Year', 'Week_Number']).agg({
    '在庫数（箱）': 'mean',
    '入庫数（箱）': 'mean',
    '出庫数（箱）': 'mean'
}).reset_index()

# 集計したデータを新しいCSVファイルに保存します。
with open('kari1.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_inventory_aggregated.to_csv(f)

weekly_inventory_aggregated.head(300)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # This is added back by InteractiveShellApp.init_path()


,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,019120LC030,2023,36,11.900000,0.216667,0.208333
1,019120LC030,2023,37,11.475000,0.233333,0.200000
2,019120LC030,2023,38,11.200000,0.225000,0.208333
3,019120LC030,2023,39,11.083333,0.233333,0.208333
4,019120LC040,2023,36,3.083333,0.025000,0.033333
...,...,...,...,...,...,...
295,35409ECB010,2023,39,2.425000,0.075000,0.075000
296,354110LC030,2023,36,43.983333,1.291667,1.350000
297,354110LC030,2023,37,41.625000,1.308333,1.300000
298,354110LC030,2023,38,41.341667,1.400000,1.358333


In [88]:
# '納入日'を日時型に変換
pre_zaikodata['納入日'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')
# 日時型に変換できなかった行を削除
pre_zaikodata.dropna(subset=['納入日'], inplace=True)

# 'Week_Number' 列を追加（年と週の番号を取得）
pre_zaikodata['Year'] = pre_zaikodata['納入日'].dt.year
pre_zaikodata['Week_Number'] = pre_zaikodata['納入日'].dt.week

# '品番'と'Week_Number'でグループ化し、'収容数'を合計
weekly_aggregated = pre_zaikodata.groupby(['品番', '仕入先名','Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C']).agg({'収容数': 'mean', '納入便': 'mean', '社内LT（検収〜回収LT）': 'mean','日量数（箱数）':'mean'})
# '品番'と'Week_Number'でグループ化し、各グループの行数（かんばん数）をカウント
weekly_kanban_count = pre_zaikodata.groupby(['品番','仕入先名','Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C']).size().reset_index(name='かんばん数')

# 2つの集計データセットを結合
merged_weekly_data = pd.merge(weekly_aggregated, weekly_kanban_count, on=['品番', 'Year', 'Week_Number', '基準在庫日数', '基準在庫枚数','箱種類','A','B','C'], how='outer')

# 集計したデータをCSVファイルに保存
with open('kari2.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_weekly_data.to_csv(f)

merged_weekly_data.head(30)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


,品番,Year,Week_Number,基準在庫日数,基準在庫枚数,箱種類,A,B,C,収容数,納入便,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,かんばん数
0,019128GA010,2023,35,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.549451,3.782967,17.913684,石黒ゴム工業（株）,91
1,019128GA010,2023,36,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.517647,3.542588,16.656471,石黒ゴム工業（株）,85
2,019128GA010,2023,37,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.500000,3.549348,16.140435,石黒ゴム工業（株）,92
3,019128GA010,2023,38,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.523810,3.583333,16.145476,石黒ゴム工業（株）,84
4,019128GA010,2023,39,1.07,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50.0,1.423077,2.525962,14.479231,石黒ゴム工業（株）,52
5,01912ECB010,2023,35,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.517241,4.024828,6.305556,石黒ゴム工業（株）,29
6,01912ECB010,2023,36,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.483871,3.754194,5.518710,石黒ゴム工業（株）,31
7,01912ECB010,2023,37,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.466667,3.608667,5.328667,石黒ゴム工業（株）,30
8,01912ECB010,2023,38,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.500000,3.608667,5.469111,石黒ゴム工業（株）,30
9,01912ECB010,2023,39,0.85,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300.0,1.562500,2.767500,4.398333,石黒ゴム工業（株）,16


In [89]:
# 2つの集計データセットを結合
merged_data = pd.merge(weekly_inventory_aggregated, merged_weekly_data, on=['品番', 'Year', 'Week_Number'], how='inner')
# '基準在庫日数' から '社内LT（検収～回収LT）' を引いた新しい列 'LT_差分' を追加
merged_data['実績LT/設計値LT'] = merged_data['社内LT（検収〜回収LT）']/merged_data['基準在庫日数']
merged_data['順立装置在庫量/設計値MIN']=merged_data['在庫数（箱）']/(0.1*(merged_data['日量数（箱数）']*merged_data['A']*(1+merged_data['C'])/merged_data['B']))

# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
pitch = pd.read_csv('不等ピッチ情報.csv', encoding='Shift_JIS')

longi = len(merged_data.loc[:,"品番"])
longj = len(pitch.loc[:,"仕入先名"])

for i in range(longi):
    for j in range(longj):
        if (merged_data.loc[i, '仕入先名'] == pitch.loc[j,'仕入先名']):
            merged_data.loc[i,"不等ピッチ"] = pitch.loc[j,"不等ピッチ係数日"]
            
merged_data['順立装置在庫量/設計値MAX'] = merged_data['在庫数（箱）']/(merged_data['順立装置在庫量/設計値MIN'] + merged_data['日量数（箱数）']/merged_data['B'] + merged_data['日量数（箱数）']*merged_data.loc[i,"不等ピッチ"])

#S = int(str(df_design.loc[j,'収容数']).replace(',', ''))
#df_merge.loc[count,'最小値'] = 0.1 * ((daily/S)  * A * (1 + C) / B) 
#df_merge.loc[count,'最大値'] = df_merge.loc[count,'最小値'] + (daily / S)/B + (daily/S)*df_design.loc[j,'不等ピッチ']

# 集計したデータをCSVファイルに保存
with open('kari5.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)

merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,収容数,納入便,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,かんばん数,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX
0,019128GA010,2023,36,40.883333,0.725000,0.708333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50.0,1.517647,3.542588,16.656471,石黒ゴム工業（株）,85,3.310830,10.579748,0.29,2.071006
1,019128GA010,2023,37,40.166667,0.733333,0.683333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50.0,1.500000,3.549348,16.140435,石黒ゴム工業（株）,92,3.317148,10.726612,0.29,2.048917
2,019128GA010,2023,38,36.766667,0.616667,0.650000,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50.0,1.523810,3.583333,16.145476,石黒ゴム工業（株）,84,3.348910,9.815568,0.29,1.966597
3,019128GA010,2023,39,39.841667,0.700000,0.683333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,50.0,1.423077,2.525962,14.479231,石黒ゴム工業（株）,52,2.360712,11.860528,0.29,2.009759
4,01912ECB010,2023,36,12.775000,0.250000,0.233333,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,300.0,1.483871,3.754194,5.518710,石黒ゴム工業（株）,31,4.416698,9.977813,0.29,0.981703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,9011906A065,2023,38,1.623853,0.110092,0.064220,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,1000.0,3.785714,1.939167,2.411750,（株）青山製作所,14,3.658805,5.976116,0.08,0.249875
296,9011906A065,2023,39,2.975000,0.108333,0.091667,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,1000.0,2.875000,1.751250,2.330250,（株）青山製作所,8,3.304245,11.331542,0.08,0.251343
297,9014908A008,2023,36,4.683333,0.166667,0.133333,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,800.0,2.956522,2.540000,3.349602,（株）青山製作所,23,4.884615,12.409846,0.08,0.356538
298,9014908A008,2023,37,3.425532,0.117021,0.117021,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,800.0,3.384615,1.955000,3.010625,（株）青山製作所,13,3.759615,10.098943,0.08,0.318617


In [99]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_組立稼働率実績.csv'

# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
AVAILABLE_RATE = pd.read_csv(new_file_path, encoding='cp932')

# データフレームのインデックスを日付型に変換
AVAILABLE_RATE['LINE_DATE'] = pd.to_datetime(AVAILABLE_RATE['LINE_DATE'])
#AVAILABLE_RATE.set_index('LINE_DATE', inplace=True)
# 'WEEK_NUMBER'列を追加し、'LINE_DATE'の週番号を計算
AVAILABLE_RATE['Week_Number'] = AVAILABLE_RATE['LINE_DATE'].dt.week
AVAILABLE_RATE['WEEKDAY'] = AVAILABLE_RATE['LINE_DATE'].dt.weekday

# 土日を除外
AVAILABLE_RATE2 = AVAILABLE_RATE[(AVAILABLE_RATE['WEEKDAY'] != 5) & (AVAILABLE_RATE['WEEKDAY'] != 6)]

# 'AVAILABLE_RATE'列を週単位でリサンプリングし、平均を計算
weekly_average_available_rate = AVAILABLE_RATE2.groupby('Week_Number')['AVAILABLE_RATE'].mean().reset_index()
weekly_average_available_rate = AVAILABLE_RATE2.groupby('Week_Number')['KAKO_CNT'].mean().reset_index()

# 集計したデータを新しいCSVファイルに保存します。
with open('kari4.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_average_available_rate.to_csv(f)

weekly_average_available_rate.head(300)

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # This is added back by InteractiveShellApp.init_path()


,Week_Number,KAKO_CNT
0,35,424.5
1,36,469.4
2,37,445.7
3,38,430.5
4,39,454.5


In [100]:
# 2つの集計データセットを結合
merged_data = pd.merge(merged_data, weekly_average_available_rate, on=['Week_Number'], how='inner')
# 集計したデータを新しいCSVファイルに保存します。
with open('kari6.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)
merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,社内LT（検収〜回収LT）,日量数（箱数）,仕入先名,かんばん数,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,AVAILABLE_RATE,KAKO_CNT
0,019128GA010,2023,36,40.883333,0.725000,0.708333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.542588,16.656471,石黒ゴム工業（株）,85,3.310830,10.579748,0.29,2.071006,95.285,469.4
1,01912ECB010,2023,36,12.775000,0.250000,0.233333,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,3.754194,5.518710,石黒ゴム工業（株）,31,4.416698,9.977813,0.29,0.981703,95.285,469.4
2,01912ECB040,2023,36,29.358333,0.708333,0.700000,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,2.393721,16.821628,愛産樹脂工業（株）,86,9.574884,5.593824,0.00,1.262369,95.285,469.4
3,01912ECB060,2023,36,2.441667,0.033333,0.041667,0.78,1.0,ＴＰ３３１フタナシ,1.0,...,5.427500,1.006875,新光ゴム工業（株）,4,6.958333,5.705870,0.00,0.361028,95.285,469.4
4,1040043104R,2023,36,3.808333,0.066667,0.066667,0.75,3.0,AW131ﾌﾀﾅｼ,1.0,...,5.081429,1.833714,（株）ギフ加藤製作所,8,6.775238,8.653505,0.12,0.394154,95.285,469.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,G1144ECE010,2023,37,65.633333,2.716667,2.375000,0.58,2.0,ＴＰ３３１フタアリ,1.0,...,1.914632,57.202837,矢崎総業（株）,326,3.301089,4.112469,0.00,1.022717,90.551,445.7
296,G1163ECB010,2023,37,2.611111,0.342593,0.314815,0.54,1.0,ＴＰ３３１フタナシ,1.0,...,0.994634,7.930793,（株）青山製作所,44,1.841915,2.922223,0.08,0.562671,90.551,445.7
297,G1163ECE010,2023,37,10.500000,0.750000,0.750000,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,1.374062,17.912187,（株）青山製作所,106,2.592571,5.202897,0.08,1.155895,90.551,445.7
298,G117362010,2023,37,54.383333,1.766667,1.908333,0.85,1.0,ＴＰ３３１．５フタナシ,1.0,...,2.329462,45.019444,多摩川精機販売（株）,260,2.740543,2.982708,0.00,1.082188,90.551,445.7


In [101]:
# 品番と週番号でデータを並べ替え
merged_data2 = merged_data.sort_values(by=['品番', 'Week_Number'])
# 同じ品番ごとに在庫数の週間差分を計算
merged_data2['先週からの順立装置在庫量の増加率'] = merged_data2.groupby('品番')['在庫数（箱）'].pct_change()
merged_data2['先週からの社内LTの増加率'] = merged_data2.groupby('品番')['社内LT（検収〜回収LT）'].pct_change()
merged_data2['先週からかんばん増加数'] = merged_data2.groupby('品番')['かんばん数'].pct_change()
# 差分がNaN（最初の週）である場合は0に置き換え
merged_data2['先週からの順立装置在庫量の増加率'].fillna(0, inplace=True)
merged_data2['先週からの社内LTの増加率'].fillna(0, inplace=True)
merged_data2['先週からかんばん増加数'].fillna(0, inplace=True)
merged_data2.head(30)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,かんばん数,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,AVAILABLE_RATE,KAKO_CNT,先週からの順立装置在庫量の増加率,先週からの社内LTの増加率,先週からかんばん増加数
0,019128GA010,2023,36,40.883333,0.725000,0.708333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,85,3.310830,10.579748,0.29,2.071006,95.285,469.4,0.000000,0.000000e+00,0.000000
151,019128GA010,2023,37,40.166667,0.733333,0.683333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,92,3.317148,10.726612,0.29,2.048917,90.551,445.7,-0.017530,1.908094e-03,0.082353
306,019128GA010,2023,38,36.766667,0.616667,0.650000,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,84,3.348910,9.815568,0.29,1.966597,94.160,430.5,-0.084647,9.575141e-03,-0.086957
450,019128GA010,2023,39,39.841667,0.700000,0.683333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,52,2.360712,11.860528,0.29,2.009759,98.060,454.5,0.083636,-2.950805e-01,-0.380952
1,01912ECB010,2023,36,12.775000,0.250000,0.233333,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,31,4.416698,9.977813,0.29,0.981703,95.285,469.4,0.000000,0.000000e+00,0.000000
152,01912ECB010,2023,37,12.900000,0.233333,0.233333,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,30,4.245490,10.434777,0.29,0.965168,90.551,445.7,0.009785,-3.876382e-02,-0.032258
307,01912ECB010,2023,38,11.633333,0.208333,0.216667,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,30,4.245490,9.168524,0.29,0.955389,94.160,430.5,-0.098191,2.220446e-16,0.000000
451,01912ECB010,2023,39,14.300000,0.233333,0.241667,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,16,3.255882,14.013929,0.29,0.870200,98.060,454.5,0.229226,-2.330962e-01,-0.466667
2,01912ECB040,2023,36,29.358333,0.708333,0.700000,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,86,9.574884,5.593824,0.00,1.262369,95.285,469.4,0.000000,0.000000e+00,0.000000
153,01912ECB040,2023,37,26.675000,0.766667,0.675000,0.25,1.0,ＴＰ３４２フタナシ,1.0,...,93,9.093763,5.263257,0.00,1.195141,90.551,445.7,-0.091399,-5.024816e-02,0.081395


In [102]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_使用工程.csv'
# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
step = pd.read_csv(new_file_path, encoding='cp932')

longi = len(merged_data2.loc[:,"品番"])
longj = len(step.loc[:,"品番"])

for i in range(longi):
    hinban = merged_data2.loc[i,'品番'] 
    hinban2 = hinban.replace("-", "") 
    hinban3 = hinban2.replace(" ", "") 
    for j in range(longj):
        stephinban = step.loc[j,'品番'].replace('-', '')
        stephinban2 = stephinban.replace(' ', '')
        if (hinban3 == stephinban2):
            merged_data2.loc[i,"使用工程"] = step.loc[j,"使用工程"]
            
# 集計したデータを新しいCSVファイルに保存します。
with open('data.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data2.to_csv(f)
    
merged_data2.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫枚数,箱種類,A,...,実績LT/設計値LT,順立装置在庫量/設計値MIN,不等ピッチ,順立装置在庫量/設計値MAX,AVAILABLE_RATE,KAKO_CNT,先週からの順立装置在庫量の増加率,先週からの社内LTの増加率,先週からかんばん増加数,使用工程
0,019128GA010,2023,36,40.883333,0.725000,0.708333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.310830,10.579748,0.29,2.071006,95.285,469.4,0.000000,0.000000,0.000000,8.ファイナル後半
151,019128GA010,2023,37,40.166667,0.733333,0.683333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.317148,10.726612,0.29,2.048917,90.551,445.7,-0.017530,0.001908,0.082353,8.ファイナル後半
306,019128GA010,2023,38,36.766667,0.616667,0.650000,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,3.348910,9.815568,0.29,1.966597,94.160,430.5,-0.084647,0.009575,-0.086957,8.ファイナル後半
450,019128GA010,2023,39,39.841667,0.700000,0.683333,1.07,1.0,ＴＰ３４２フタナシ,1.0,...,2.360712,11.860528,0.29,2.009759,98.060,454.5,0.083636,-0.295081,-0.380952,8.ファイナル後半
1,01912ECB010,2023,36,12.775000,0.250000,0.233333,0.85,1.0,ＴＰ３３２フタナシ,1.0,...,4.416698,9.977813,0.29,0.981703,95.285,469.4,0.000000,0.000000,0.000000,8.ファイナル後半
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,9011906A065,2023,38,1.623853,0.110092,0.064220,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,3.658805,5.976116,0.08,0.249875,94.160,430.5,-0.207876,-0.171296,0.000000,2.MGケース
522,9011906A065,2023,39,2.975000,0.108333,0.091667,0.53,1.0,ＴＰ３３１フタナシ,1.0,...,3.304245,11.331542,0.08,0.251343,98.060,454.5,0.832062,-0.096906,-0.428571,2.MGケース
75,9014908A008,2023,36,4.683333,0.166667,0.133333,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,4.884615,12.409846,0.08,0.356538,95.285,469.4,0.000000,0.000000,0.000000,7.ファイナル前半
226,9014908A008,2023,37,3.425532,0.117021,0.117021,0.52,1.0,ＴＰ３３１フタナシ,1.0,...,3.759615,10.098943,0.08,0.318617,90.551,445.7,-0.268570,-0.230315,-0.434783,7.ファイナル前半


In [103]:
step.head(300)

,品番,使用工程
0,34989ECE030,NaN
1,9036340A010,NaN
2,35145ECE020,2.MGケース
3,33490ECB010,7.ファイナル前半
4,33490ECE010,7.ファイナル前半
...,...,...
143,9036652A004,1.デフ・大物ギヤ・Brg
144,9036628A007,1.デフ・大物ギヤ・Brg
145,9036630A017,1.デフ・大物ギヤ・Brg
146,9036628A006,1.デフ・大物ギヤ・Brg
